<a href="https://colab.research.google.com/github/szn5400/CO2_all/blob/main/gradienttape_for_grad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
X = [[2,2],[3,3],[4,4],[5,5],[6,6],[7,7]]
Y = [[4,4],[6,6],[8,8],[10,10],[12,12],[14,14]]

In [13]:
size_input = 2
size_hidden = 1
size_output = 2
number_of_train_examples = 2
number_of_test_examples = 2

In [14]:
import os
import numpy as np
import time
import tensorflow as tf

np.random.seed(43)
tf.random.set_seed(43)


In [49]:
class MLP(object):
  def __init__(self, size_input, size_hidden, size_output, device=None):
    """
    size_input: int, size of input layer
    size_hidden1: int, size of hidden layer 1
    size_hidden2: int, size of hodden layer 2
    size_output: int, size of output layer
    device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
    """
    self.size_input, self.size_hidden, self.size_output, self.device =\
    size_input, size_hidden, size_output, device
    
    # Initialize weights between input layer and hidden layer
    self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden]))
    # Initialize biases for hidden layer
    self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden]))
     # Initialize weights between hidden layer and output layer
    self.W2 = tf.Variable(tf.random.normal([self.size_hidden, self.size_output]))
    # Initialize biases for output layer
    self.b2 = tf.Variable(tf.random.normal([1, self.size_output]))
    
    # Define variables to be updated during backpropagation
    self.variables = [self.W1, self.W2, self.b1, self.b2]
    
  def forward(self, X):
    """
    forward pass
    X: Tensor, inputs
    """
    if self.device is not None:
      with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
        self.y = self.compute_output(X)
    else:
      self.y = self.compute_output(X)
      
    return self.y
  
  def loss(self, y_pred, y_true):
    '''
    y_pred - Tensor of shape (batch_size, size_output)
    y_true - Tensor of shape (batch_size, size_output)
    '''
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, self.size_output)), dtype=tf.float32)
    loss1 =  tf.losses.mean_squared_error(y_true_tf, y_pred_tf)

    '''
    with tf.GradientTape() as tape:
      predicted_out1 = self.forward(self.X)[:,0]
      print('\n\n we are here')
      print('predicted_out1', predicted_out1)
      print(self.X[:,1])
      #grad_gas_with_time = tape.gradient(predicted_out1,self.X[:,1])
      grad_gas_with_time = tape.gradient(self.forward(self.X)[:,0],self.X[:,1])
      print('gard values',grad_gas_with_time)
    '''
    with tf.GradientTape(persistent=True) as tape:
      predicted_out1 = tf.cast(self.forward(self.X), tf.float64)
      print('\n\n we are here')
      print('predicted_out1', predicted_out1)
      print(self.X[:,1])
      grad_gas_with_time = tape.gradient(predicted_out1,tf.cast(self.X, tf.float64))
    #grad_gas_with_time = tape.gradient(self.forward(self.X),self.X)
    print('gard values',grad_gas_with_time)

    return loss1

  def accuracy(self, y_pred, y_true):
    y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
    y_pred_tf = tf.cast(tf.reshape(y_pred, (-1, self.size_output)), dtype=tf.float32)
    correct_pred = tf.reduce_sum(tf.abs(y_true_tf-y_pred_tf))  
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')
    return accuracy
  
  def backward(self, X_train, y_train):
    """
    backward pass
    """
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)
    with tf.GradientTape() as tape:
      predicted = self.forward(X_train)
      current_loss = self.loss(predicted, y_train)
    grads = tape.gradient(current_loss, self.variables)
    optimizer.apply_gradients(zip(grads, self.variables))
        
        
  def compute_output(self, X):
    """
    Custom method to obtain output tensor during forward pass
    """
    # Cast X to float32
    X_tf = tf.cast(X, dtype=tf.float32)
    self.X = tf.Variable(X_tf)
    # Compute values in hidden layer1
    what1 = tf.matmul(X_tf, self.W1) + self.b1
    hhat1 = tf.nn.relu(what1)

    # Compute output
    output = tf.matmul(hhat1, self.W2) + self.b2
    output = tf.nn.softmax(output)
    print(output)
    return output

In [50]:
# Set number of epochs
NUM_EPOCHS = 2

In [51]:
# Initialize model using CPU
mlp_on_cpu = MLP(size_input, size_hidden, size_output, device='gpu')

# Array to store accuracy and loss
loss_with_epoch = []
acc_with_epoch = []

time_start = time.time()
for epoch in range(NUM_EPOCHS):
  ac = 0
  count = 0
  loss_total = tf.zeros([1,1], dtype=tf.float32)
  lt = 0
  train_ds = tf.data.Dataset.from_tensor_slices((X, Y)).shuffle(25, seed=epoch*(1234)).batch(2)
  for inputs, outputs in train_ds:
    inputs = tf.Variable(inputs)
    print('input output', inputs, outputs)
    preds = mlp_on_cpu.forward(inputs)
    loss_total = loss_total + mlp_on_cpu.loss(preds, outputs)
    lt = lt + mlp_on_cpu.loss(preds, outputs)
    mlp_on_cpu.backward(inputs, outputs)
    ac = ac+mlp_on_cpu.accuracy(preds, outputs)
    #ac = mlp_on_cpu.accuracy(preds, outputs)
    count += 1
  print('Number of Epoch = {} - Average celoss:= {}- Acc:= {} '.format(epoch + 1, np.sum(loss_total) / len(X), ac/count))
  loss_with_epoch.append(np.sum(loss_total) / len(X))
  acc_with_epoch.append(ac/count)
time_taken = time.time() - time_start

print('\nTotal time taken (in seconds): {:.2f}'.format(time_taken))
#For per epoch_time = Total_Time / Number_of_epochs

input output <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32, numpy=
array([[2, 2],
       [4, 4]], dtype=int32)> tf.Tensor(
[[4 4]
 [8 8]], shape=(2, 2), dtype=int32)
tf.Tensor(
[[0.17756766 0.8224323 ]
 [0.17756766 0.8224323 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.17756766 0.8224323 ]
 [0.17756766 0.8224323 ]], shape=(2, 2), dtype=float32)


 we are here
predicted_out1 tf.Tensor(
[[0.17756766 0.82243228]
 [0.17756766 0.82243228]], shape=(2, 2), dtype=float64)
tf.Tensor([2. 4.], shape=(2,), dtype=float32)
gard values None
tf.Tensor(
[[0.17756766 0.8224323 ]
 [0.17756766 0.8224323 ]], shape=(2, 2), dtype=float32)


 we are here
predicted_out1 tf.Tensor(
[[0.17756766 0.82243228]
 [0.17756766 0.82243228]], shape=(2, 2), dtype=float64)
tf.Tensor([2. 4.], shape=(2,), dtype=float32)
gard values None
tf.Tensor(
[[0.17756766 0.8224323 ]
 [0.17756766 0.8224323 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[0.17756766 0.8224323 ]
 [0.17756766 0.8224323 ]], shape=(2, 2), dtype=float32)